In [1]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

ModuleNotFoundError: No module named 'fastbook'

In [2]:
#hide
from fastbook import *
from fastai.vision.widgets import *

ModuleNotFoundError: No module named 'fastbook'

# 2.2 데이터 수집
### 세 종류의 곰을 구분하는 곰 탐지기 프로젝트.

* 빙 이미지 검색 사용 

In [3]:
import os

In [1]:
key = os.environ.get('AZURE_SEARCH_KEY', 'XXX')

NameError: name 'os' is not defined

key 를 셋팅하기 위해서는, 터미널에서 <br>
export AZURE_SEARCH_KEY=your_key_here
<br>
명령어에 발급받은 AZURE 이미지 검색 API 키를 넣어야 함.

In [1]:
search_images_bing 
# 위 함수 사용 가능!
# 이 함수는 책 저장소의 utils라는 파일이 제공하는 편의 기능.
# 함수가 정의된 위치를 파악하기 어렵다면 위 같이 주피터 노트북 셀에 함수 이름을 적고 실행해 찾음.

NameError: name 'search_images_bing' is not defined

In [ ]:
results = search_images_bing(key, 'grizzly bear')
ims = results.attrgot('contentUrl')
len(ims)

# 이 코드로 회색곰 이미지 150장의 URL을 성공적으로 다운로드 함.

In [ ]:
#위에서 다운로드한 이미지 하나를 살펴보기

dest = 'images/grizzly.jpg'
download_url(ims[0], dest)

im = Image.alpha_compositeopen(dest)
im.to_thumb(128,128)


In [ ]:
# 각 검색어에서 얻은 모든 URL의 이미지를 fastai가 제공하는 download_images 함수로 다운로드.
# 검색어별 이미지는 검색어가 이름인 별도의 폴더에 저장하도록 코드 작성.

bear_types = 'grizzly', 'black', 'teddy'
path = Path('bears')

if not path.exists():
    path.mkdir()
    for o in bear_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{o} bear')
        download_images(dest, urls=results.attrgot('contentUrl'))

=> 위 코드 생성 시 폴더와 이미지 파일이 생김

In [ ]:
fns = get_image_files(path)
fns

In [ ]:
# 인터넷에서 파일을 다운로드하다 보면 가끔 파일에 오류가 발생하곤 함.
# 파일의 오류 여부는 다음과 같이 verify_images 함수로 검사할 수 있다.

failed = verify_images(fns)
failed

상기 코드에는 오류가 발생한 파일이 없지만, 오류 파일을 삭제하려면 unlink 함수를 사용함. <br>
**컬렉션 collection** 자료형을 반환하는 여러 fastai 함수처럼 verify_images 또한 L이라는 컬렉션 객체를 반환함. <br>
이 객체에 있는 map 메서드는 호출 시 인자로 전달된 함수를 컬렉션에 포함된 각 요소에 적용. <br>
즉, 다음 코드는 unlink 함수를 사용해 오류가 발생한 모든 파일(요소)을 삭제함.

In [ ]:
failed.map(Path.unlink);

# 2.3 데이터에서 DataLoaders까지

* 지금까지 모델 학습에 사용할 이미지를 다운로드하고 파일 오류를 검증했습니다.
* 이제는 다운로드한 데이터를 DataLoaders 객체로 만들 차례.
* DataLoaders는 단순히 전달받은 여러 DataLoader 객체를 목적에 맞게 학습용과 검증용으로 나누어 저장하는 클래스.
* 다음 코드 네 줄은 DataLoaders의 주요 기능을 보여 줌.

In [ ]:
class DataLoaders(GetAttr):
    def __init__(self, *loaders): self.loaders = loaders
    def __getitem__(self, i): return self.loaders[i]
    train,valid = add_props(lambda i,self: self[i])

다운로드한 데이터로 DataLoaders로 만들려면 적어도 다음과 같은 네 가지 정보가 필요합니다.
* 작업 데이터 유형
* 데이터 목록을 가져오는 방법
* 각 데이터에 레이블을 지정하는 방법
* 검증용 데이터셋을 만드는 방법
지금까지는 미리 정의된 네 개의 특정 조합으로 구현된 여러 가지 **팩토리 메서드 factory method**를 접했습니다. <br>
그런데 딱 들어맞는 메서드가 없는 경우, fastai는 **데이터블록 data block**이라는 유연한 시스템을 제공. <br>
데이터블록 API를 사용해 DataLoaders 생성에 관련된 모든 단계를 완전히 사용자의 상황에 맞게 정의할 수 있음. <br>
다음 코드는 데이터블록 API를 사용해 우리가 다운로드한 데이터셋을 DataLoaders로 만드는 방법.

In [ ]:
bears = DataBlock(blocks=(ImageBlock, CategoryBlock), # 독립변수와 종속변수의 데이터 유형을 튜플tuple 형태로 지정.
                  get_items=get_image_files,
                  splitter=RandomSplitter(valid_pct=0.2, seed=42),
                  get_y=parent_label,
                  item_tfms=Resize(128))